In [1]:
import pandas as pd
import re
import numpy as np

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

C:\Users\War\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv("DF_BOOTCAMPS_5158.csv")

In [3]:
#CONSERVAR A TODOS AQUELLOS QUE NO TENGAN MISSING VALUES EN EDUCATION Y CERTIFICATION. 

dfe = df[df["education"].notna()] #DATAFRAME DE EDUCATION
dfe = dfe.reset_index(drop = True) #REINICIAR INDICE

dfc = df[df["certifications"].notna()] #DATAFRAME DE CERTIFICACIÓN

In [19]:
# REGEX DE BOOTCAMPS ACTUALIZADAS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'iron.?hack',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'mind.?hub',
 'muktek|muk.?tek', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'worldtech$'] 

In [20]:
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'Iron Hack',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'MindHub ',
 'Muktek Academy',
 'Noukod',
 'Plataforma 5',
 'World Tech Inc (a.k.a. World Tech Makers –WTM–)']

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE EDUCACIÓN

In [21]:
#CONVERTIR EN STRING
dfe["education"] =  dfe["education"].str.lower()

In [22]:
#CONVERTIR EN DICCIONARIOS LA EDUCACIÓN
L = []

for i in dfe["education"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]

"{'education_org': 'university of oxford', 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user', 'end': '2013', 'start': '2013'}"

In [23]:
K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

{'education': 'university of oxford',
 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user',
 'start': '2013',
 'end': '2013'}

In [24]:
#CREAR COLUMNA LIMPIA DE EDUCACIÓN
dfe["educacion"] = K

In [25]:
#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
dfe.educacion[0][2]

{'education': 'vrije universiteit amsterdam',
 'degrees': '(pre)-master, organization science',
 'summary': '(schakeljaar tussen hbo-opleiding en masterstudie)\\n afstudeeronderzoek bij het ministerie van sociale zaken en werkgelegenheid(szw)\\n onderzoek op welke manier(en) de invoering van ‘het nieuwe werken’ bij het ministerie van szw, invloed heeft (gehad) op de betrokkenheid van de werknemers bij het uitvoeren hun werkzaamheden.\\n activities and societies: lid van faculteitsvereniging eos',
 'start': '2011',
 'end': '2012'}

In [26]:
#LOOP PARA BUSCAR SI EL INDIVIDUO TIENE FORMACIÓN EN ALGÚN BOOTCAMP
c = []
for boot in range(len(boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfe.educacion)): #PARA CADA INDIVIDUO
        for h in range(len(dfe.educacion[i])): #DENTRO LA EDUCACIÓN DE CADA INDIVIDUO
            if "education" in dfe.educacion[i][h]: #SI TIENE EL CAMPO EDUCACIÓN
                A = re.findall(boots[boot], dfe.educacion[i][h]["education"]) #REVISA SI ESTÁ EN UN BOOTCAMP
                if len(A) > 0 : #SI ESA REVISIÓN ES NO VACÍA
                    c.append((dfe.id[i], boots[boot], lista_boots[boot], dfe.educacion[i][h]['education']))
                    # c.append((dfe.id[i], h)) #AÑADE EL ID DE ESE INDIVIDUO Y EL LUGAR DONDE ESTÁ ESA EDUCACIÓN
                    # print(f"{dfe['job_title'][i]}---{A}---{boots[boot]}--{dfe.educacion[i][h]['education']} \n \n")

#CREAR DATAFRAME
c = list(set(c))
educ_boots = pd.DataFrame.from_records(c, columns=['id', 'regex_bootcamp', 'bootcamp', "educ_title"])

#HACER MERGE CON EL RESTO DE COLUMNAS
# educ_boots = pd.merge(educ_boots, dfe, "left", "id")
educ_boots.to_excel("2_educ_regex.xlsx", index = False)

In [27]:
#EDUC PARTUL
k = []
## se utiliza los nombres normales
for boot in range(len(lista_boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfe.educacion)): #PARA CADA INDIVIDUO
        for h in range(len(dfe.educacion[i])): #DENTRO LA EDUCACIÓN DE CADA INDIVIDUO
            if "education" in dfe.educacion[i][h]: #SI TIENE EL CAMPO EDUCACIÓN
                ## se hace un aproach a fuzzy
 
                bootCAMP = lista_boots[boot]
                bootCAMP = bootCAMP.lower()
                bootCAMP = unidecode(bootCAMP)
                norEdu =  unidecode(dfe.educacion[i][h]["education"])
                
                wordLen = len(dfe.educacion[i][h]["education"])
                ratio = fuzz.partial_ratio(bootCAMP,norEdu)

                if ratio>85 and wordLen >=5:
                    k.append((dfe.id[i], boots[boot], lista_boots[boot], dfe.educacion[i][h]['education'],ratio))


#CREAR DATAFRAME
k = list(set(k))
educ_fp = pd.DataFrame.from_records(k, columns=['id', 'regex_bootcamp', 'bootcamp', "educ_title", "ratio"])

#HACER MERGE CON EL RESTO DE COLUMNAS
# educ_boots = pd.merge(educ_boots, dfe, "left", "id")
educ_fp.to_excel("2_educ_fp.xlsx", index = False)

In [28]:
#EDUC PARTIAL

p = []
## se utiliza los nombres normales
for boot in range(len(lista_boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfe.educacion)): #PARA CADA INDIVIDUO
        for h in range(len(dfe.educacion[i])): #DENTRO LA EDUCACIÓN DE CADA INDIVIDUO
            if "education" in dfe.educacion[i][h]: #SI TIENE EL CAMPO EDUCACIÓN
                ## se hace un aproach a fuzzy
                bootCAMP = lista_boots[boot]
                bootCAMP = bootCAMP.lower()
                bootCAMP = unidecode(bootCAMP)
                norEdu =  unidecode(dfe.educacion[i][h]["education"])
                ## se hace un aproach a fuzzy
                wordLen = len(dfe.educacion[i][h]["education"])
                # ratio = fuzz.partial_ratio(bootCAMP,norEdu)
                #ratio = fuzz.token_sort_ratio(bootCAMP,norEdu)
                ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
                #print(ratio)
                if ratio>85 and wordLen >=5:
                    p.append((dfe.id[i], boots[boot], lista_boots[boot], dfe.educacion[i][h]['education'],ratio))

#CREAR DATAFRAME
p = list(set(p))
educ_fs = pd.DataFrame.from_records(p, columns=['id', 'regex_bootcamp', 'bootcamp', "educ_title", "ratio"])

#HACER MERGE CON EL RESTO DE COLUMNAS
# educ_boots = pd.merge(educ_boots, dfe, "left", "id")
educ_fs.to_excel("2_educ_fs.xlsx", index = False)